In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants as sc
import pickle

from DeepFMKit.experiments import Experiment
from DeepFMKit.factories import StandardWDFMIExperimentFactory
from DeepFMKit.waveforms import second_harmonic_distortion # Even if unused, good practice

# --- 1. Define the Reusable Analysis and Plotting Function ---
def plot_bias_landscape(results_file: str):
    """
    Loads and plots the 2D bias landscape from an experiment's results file.
    """
    try:
        with open(results_file, 'rb') as f:
            results = pickle.load(f)
    except FileNotFoundError:
        print(f"Results file not found: {results_file}. Please run the experiment first.")
        return

    # Extract axes and results
    m_main_range = results['axes']['m_main']
    m_witness_range = results['axes']['m_witness']
    fitter_name = results['fitter_name']
    
    # Get the grid of fitted 'm' values. Shape is (len(m_witness), len(m_main))
    # Note: my aggregation logic now makes the first axis the outer loop.
    m_fit_grid = results['wdfmi_fit']['m']['mean']

    # Create the grid of true m_main values for subtraction.
    # It must have the same shape as m_fit_grid for element-wise subtraction.
    m_true_grid = np.tile(m_main_range, (len(m_witness_range), 1))
    
    bias_grid = m_fit_grid - m_true_grid

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 9))
    max_abs_bias = 0.01 # Use a sensitive color scale for residual bias
    
    # The grid shapes now match the axes for pcolormesh.
    # X axis: m_main_range, Y axis: m_witness_range, C (color) axis: bias_grid
    im = ax.pcolormesh(
        m_main_range, m_witness_range, np.abs(bias_grid),
        vmin=0, vmax=max_abs_bias, cmap='magma', shading='gouraud'
    )
    
    ax.set_title(f'W-DFMI Residual Bias Landscape\n(Fitter: {fitter_name})', fontsize=16)
    ax.set_xlabel(r'Main Modulation Depth, $m_{\rm main}$ (rad)', fontsize=14)
    ax.set_ylabel(r'Witness Modulation Depth, $m_{\rm witness}$ (rad)', fontsize=14)
    
    cbar = fig.colorbar(im, ax=ax, label=r'Absolute Residual Bias, $|\delta m|$ (rad)')
    
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

# --- 2. Declaratively Define and Run the Experiment ---
if __name__ == "__main__":
    fitter_to_test = 'wdfmi_ortho'
    exp = Experiment(description=f"W-DFMI Residual Bias")
    
    # Define the 2D parameter grid
    exp.add_axis('m_witness', np.linspace(0.1, 1.0, 5)) # Outer loop
    exp.add_axis('m_main', np.linspace(3, 25, 5))    # Inner loop
    
    # This experiment is deterministic (no stochastic variables)
    exp.n_trials = 1

    # Use the standard factory. We pass a simple cosine wave as the waveform function.
    # This tests the "intrinsic" bias of the algorithm itself.
    exp.set_config_factory(
        StandardWDFMIExperimentFactory(waveform_function=np.cos)
    )
    
    exp.add_analysis(name='wdfmi_fit', fitter_method=fitter_to_test, result_cols=['m'])
    
    # Run the experiment
    results = exp.run()

    # Save results
    results_filename = f'interaction_results_{fitter_to_test}.pkl'
    if results:
        results['fitter_name'] = fitter_to_test
        with open(results_filename, 'wb') as f:
            pickle.dump(results, f)
        print(f"Results saved to {results_filename}")
        
        # Plot the results using our dedicated function
        plot_bias_landscape(results_filename)